<a href="https://colab.research.google.com/github/younes-sadi/semantic-search-task2-LSP2431549/blob/main/Task2_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Belkacem Sadi LSP_Id: 2431549 Task_2

In [3]:
!pip install -q transformers datasets sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset

dataset = load_dataset("ag_news", split="train[:5000]")
texts = [x["text"] for x in dataset]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [10]:
import pandas as pd

df = pd.DataFrame({"text": texts})
df.to_csv("ag_news_5000.csv", index=False)


In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

encoder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = encoder.encode(texts, convert_to_numpy=True, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [16]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#  small generator model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
generator = T5ForConditionalGeneration.from_pretrained("t5-small")

query = "What are the latest innovations in AI?"
query_embedding = encoder.encode([query], convert_to_numpy=True)
D, I = index.search(query_embedding, k=3)

# Combine top 3 results into a context
context = " ".join(texts[i] for i in I[0])
prompt = f"question: {query} context: {context}"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
outputs = generator.generate(**inputs)
print("=== Answer ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


=== Answer ===
micro flyer


In [45]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#  t5-large
tokenizer = T5Tokenizer.from_pretrained("t5-large")
generator = T5ForConditionalGeneration.from_pretrained("t5-large")

query = "What are the latest news of sport "

query_embedding = encoder.encode([query], convert_to_numpy=True)

D, I = index.search(query_embedding, k=5)

# top 5
context = " ".join(texts[i] for i in I[0])

prompt = f"question: {query} context: {context}"

# 7. Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

outputs = generator.generate(
    input_ids=inputs["input_ids"],
    max_length=300,
    min_length=50,
    length_penalty=1.2,
    num_beams=4,
    early_stopping=True
)

# 9. Decode
print("=== Answer ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


=== Answer ===
Weight of drugs felt Weightlifting #39;s aggressive pursuit of drug cheaters will continue even if it jeopardizes its future in the Olympics, its top official said yesterday following six more positive doping cases in what is again becoming the Games #39; dirtiest sport


In [46]:
!git clone https://github.com/younes-sadi/semantic-search-task2-LSP2431549.git
!cd semantic-search-task2-LSP2431549


Cloning into 'semantic-search-task2-LSP2431549'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
